In [ ]:
import sys, os, git, time

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from MinimizeEnergy import MinimizeEnergy

import numpy as np

In [124]:
ti = time.time()

self = MinimizeEnergy(number_atoms=[5])
self.perform_energy_minimization()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

step epot maxF
0 -0.001 0.000
25 -0.100 0.018
50 -0.100 0.000
75 -0.100 0.001
100 -0.100 0.000
125 -0.100 0.000
150 -0.100 0.000
175 -0.100 0.000
200 -0.100 0.000
225 -0.100 0.000
250 -0.100 0.000
275 -0.100 0.001
300 -0.100 0.000
325 -0.100 0.000
350 -0.100 0.000
375 -0.100 0.000
400 -0.100 0.000
425 -0.100 0.000
450 -0.100 0.001
475 -0.100 0.000
500 -0.100 0.000
time = 0.53 s


In [172]:
ti = time.time()

forces = np.zeros((self.total_number_atoms,3))
for Ni, position_i, sigma_i, epsilon_i, neighbor_i in zip(np.arange(self.total_number_atoms-1),
                                            self.atoms_positions,
                                            self.atoms_sigma,
                                            self.atoms_epsilon,
                                            self.neighbor_lists):
    
    N_j = np.arange(self.total_number_atoms)[neighbor_i]
    positions_j = self.atoms_positions[neighbor_i]
    sigma_j = self.atoms_sigma[neighbor_i]
    epsilon_j = self.atoms_epsilon[neighbor_i]
    rij_xyz = (np.remainder(position_i - positions_j
                            + self.box_size[:3]/2., self.box_size[:3])
                            - self.box_size[:3]/2.).T
    rij = np.linalg.norm(rij_xyz, axis=0)
    for Nj, sigma_j0, epsilon_j0, rij0, rij_xyz0 in zip(N_j[rij < self.cut_off],
                                    sigma_j[rij < self.cut_off], epsilon_j[rij < self.cut_off],
                                    rij[rij < self.cut_off], rij_xyz.T[rij < self.cut_off]):
        sigma_ij = (sigma_i+sigma_j0)/2
        epsilon_ij = (epsilon_i+epsilon_j0)/2
        dU_dr = 48*epsilon_ij/rij0*((sigma_ij/rij0)**12-0.5*(sigma_ij/rij0)**6)
    
        forces[Ni] += dU_dr*rij_xyz0/rij0
        forces[Nj] -= dU_dr*rij_xyz0/rij0

print(forces)

tf = time.time()
print("time", 1000*(tf-ti))

[[ 1.31645606e-03 -1.98960504e-05  5.37146932e-04]
 [-1.41849742e-03  8.59606316e-04  1.59205583e-03]
 [-8.41297861e-04  1.38399101e-03  6.19519234e-04]
 [ 8.75400953e-04 -2.12222105e-03 -3.10891099e-03]
 [ 6.79382675e-05 -1.01480227e-04  3.60188994e-04]]
time 4.182100296020508


In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import distances
r_ij = mda.analysis.distances.self_distance_array(self.atoms_positions,
                                                    self.box_size)
x_ij = mda.analysis.distances.self_distance_array(self.atoms_positions * np.array([1, 0, 0]),
                                                    self.box_size)
y_ij = mda.analysis.distances.self_distance_array(self.atoms_positions * np.array([0, 1, 0]),
                                                    self.box_size)
z_ij = mda.analysis.distances.self_distance_array(self.atoms_positions * np.array([0, 0, 1]),
                                                    self.box_size)
vec_ij = np.vstack([x_ij, y_ij, z_ij]).T

dU_dr = 48*self.array_epsilon_ij/r_ij*((self.array_sigma_ij/r_ij)**12-0.5*(self.array_sigma_ij/r_ij)**6)

cpt = 0
forces = np.zeros((self.total_number_atoms,3))
for i in range(self.total_number_atoms):
    for j in range(i + 1, self.total_number_atoms):
        force_ij = dU_dr[cpt] * vec_ij[cpt] / r_ij[cpt]
        forces[i] += force_ij
        forces[j] -= force_ij

In [166]:
ti = time.time()

forces = np.zeros((self.total_number_atoms,3))
for Ni, position_i, sigma_i, epsilon_i, neighbor_i in zip(np.arange(self.total_number_atoms-1),
                                            self.atoms_positions,
                                            self.atoms_sigma,
                                            self.atoms_epsilon,
                                            self.neighbor_lists):
    N_j = np.arange(self.total_number_atoms)[neighbor_i]
    positions_j = self.atoms_positions[neighbor_i]
    sigma_j = self.atoms_sigma[neighbor_i]
    epsilon_j = self.atoms_epsilon[neighbor_i]
    
    rij_xyz = (np.remainder(position_i - positions_j
                            + self.box_size[:3]/2., self.box_size[:3])
                            - self.box_size[:3]/2.).T
    rij = np.linalg.norm(rij_xyz, axis=0)
    sigma_ij = (sigma_i+sigma_j)/2
    epsilon_ij = (epsilon_i+epsilon_j)/2

    dU_dr = 48*epsilon_ij/rij*((sigma_ij/rij)**12-0.5*(sigma_ij/rij)**6)

    for N_j0, force0 in zip(N_j, (dU_dr*rij_xyz/rij).T):
        forces[Ni] += force0
        forces[N_j0] -= force0

print(forces)

tf = time.time()
print("time", 1000*(tf-ti))

[[ 1.31645606e-03 -1.98960504e-05  5.37146932e-04]
 [-1.41849742e-03  8.59606316e-04  1.59205583e-03]
 [-8.41297861e-04  1.38399101e-03  6.19519234e-04]
 [ 8.75400953e-04 -2.12222105e-03 -3.10891099e-03]
 [ 6.79382675e-05 -1.01480227e-04  3.60188994e-04]]
time 2.0983219146728516


In [153]:
force0

array([-1.02157938e-05,  1.15413108e-04, -1.59481842e-04])

In [146]:
N_j

array([1, 2, 3, 4])